In [12]:
#### location of subject_1.mat:: https://datadryad.org/resource/doi:10.5061/dryad.gt413 ####

import numpy as np
import pandas as pd
from scipy.io import loadmat

In [2]:
def fMRI_initiate(file_name):
    subject = loadmat(file_name)
    time = subject['time']
    data = subject['data']
    meta = subject['meta']
    words = subject['words']
    
    result = {
        'time': time,
        'data': data,
        'meta': meta,
        'words': words
    }
    
    return result


In [3]:
## get words and its length, start time and duration
def get_words_start_time(words):
    count_records = words.shape[1]
    array_records = [];
    for i in range(count_records):
        single_row = words[0,i]
        transfer_row = np.array(["[" + single_row[0][0,0][0] + "]", len(single_row[0][0,0][0]), single_row[1][0,0], single_row[2][0,0]])
        array_records.append(transfer_row)
    return  np.array(array_records)
    #np.savetxt('foo.txt', result_array, delimiter="~", fmt="%s")

In [5]:
## fourfold data and extend time from 0,2 --> 0,0.5,1,1.5,2
def get_data_time_fourfold(data, time):
    subject_data_time =  np.concatenate((time, data), axis=1)
    subject_data_time = subject_data_time.astype(np.object)
    subject_data_time_fourfold = []
    for i in range(subject_data_time.shape[0]):
        temp = subject_data_time[i,:].copy()
        row_05 = temp.copy()
        row_05[0] += 0.5
        row_10 = temp.copy()
        row_10[0] += 1
        row_15 = temp.copy()
        row_15[0] += 1.5
        subject_data_time_fourfold.append(temp)
        subject_data_time_fourfold.append(row_05)
        subject_data_time_fourfold.append(row_10)
        subject_data_time_fourfold.append(row_15)
    subject_data_time_fourfold = np.array(subject_data_time_fourfold)
    
    for i in range(subject_data_time_fourfold.shape[0]-1):
        assert(subject_data_time_fourfold[i,0] == subject_data_time_fourfold[i+1, 0]-0.5)
    
    subject_data_time_fourfold_time = subject_data_time_fourfold[:,0:2] ## time - fourfold 0,0.5,1,1.5,2 ... 
    subject_data_time_fourfold_data = subject_data_time_fourfold[:,2:]  ## data - fourfold.
    
    assert(subject_data_time_fourfold_time.shape[0] == time.shape[0] * 4)
    assert(subject_data_time_fourfold_data.shape[0] == data.shape[0] * 4)
    assert(subject_data_time_fourfold_data.shape[1] == data.shape[1])
    
    result = {
        'fourfold_time': subject_data_time_fourfold_time,
        'fourfold_data': subject_data_time_fourfold_data
    }
    return result

In [9]:
## normalize fourfold data to be between 0 and 1 and round to 4 decimals
def data_fourfold_normalize_round(data):
    dmax, dmin = data.max(), data.min()
    result = (data - dmin) / (dmax - dmin)
    result_round = np.around(result.astype(np.double), decimals=4)
    return result_round

In [18]:
def merge_word_time_data(words, time, data):
    ## words: word_start_time
    ## time: fourfold_time,
    ## data: fourfold_data_normalized
    assert(time.shape[0] == data.shape[0])
    words.astype(np.object)
    time_data = np.concatenate((time, data), axis=1)
    result = []
    i = 0 # pointer for words [time has gap]
    j = 0 # pointer for time_data [time has no gap]
    while j < len(time_data):
        if str(words[i,2]) == str(time_data[j,0]):
            word_time_data = np.append(words[i], time_data[j])
            i += 1
            j += 1
        elif str(words[i,2]) > str(time_data[j,0]):
            word_time_data = np.append(np.ones(words.shape[1])*(-1), time_data[j])
            j += 1
        else:
            word_time_data = np.append(words[i], np.ones(time_data[j])*(-1))
            i += 1
        result.append(word_time_data)
    result = np.array(result)
    assert(result.shape[0] == time_data.shape[0])
    assert(result.shape[1] == words.shape[1] + time_data.shape[1])

    return result
    

In [11]:
def apply_gauss_window(data, window_size = 3):
    ## apply gaussian weights 
    gauss = []
    
    

SyntaxError: unexpected EOF while parsing (<ipython-input-11-6e1c4e02100c>, line 2)

In [19]:
def fMRI_process(file = 'subject_1.mat'):
    ## umbrella method to call all methods
    subject = fMRI_initiate(file)
    subject_words = subject['words']
    words_start_time = get_words_start_time(subject_words)  ## get words with start time and duration
    
    subject_time = subject['time']
    subject_data = subject['data']
    
    fourfold = get_data_time_fourfold(subject_data, subject_time) ## fourfold
    fourfold_time = fourfold['fourfold_time']
    fourfold_data = fourfold['fourfold_data']
    
    fourfold_data_norm = data_fourfold_normalize_round(fourfold_data)
    
    word_features_orig = merge_word_time_data(words_start_time, fourfold_time, fourfold_data_norm)
    
    return word_features_orig

In [20]:
word_features_orig = fMRI_process(file = 'subject_1.mat')

MemoryError: 